## Data Cleaning +	Handling Missing Entries + Duplicate Elimination

In [ ]:
import pandas as pd
import re

# 0. MEMUAT DATASET
nama_file_input = 'export_1750496855742(tokped_review).csv'

try:
    # Membaca file CSV dengan pemisah (separator) titik koma (;)
    df = pd.read_csv(nama_file_input, sep=';')
    print(f"✅ Dataset '{nama_file_input}' berhasil dimuat.")
    print(f"Bentuk data awal: {df.shape[0]} baris, {df.shape[1]} kolom.")
except FileNotFoundError:
    print(f"❌ ERROR: File '{nama_file_input}' tidak ditemukan. Pastikan file berada di folder yang sama.")
    exit()

# 1. DATA CLEANING
print("\nMemulai proses Data Cleaning...")

# 1.1 Memilih kolom yang relevan untuk analisis
kolom_relevan = [
    'Review Date',
    'Review Pros',
    'Review Cons',
    'Review Rating Overall',
    'Is Current Job'
]
# Memastikan semua kolom relevan ada di dataframe
kolom_tersedia = [col for col in kolom_relevan if col in df.columns]
df_cleaned = df[kolom_tersedia].copy()
print(f"   - Kolom yang tidak relevan dibuang. Sisa {len(df_cleaned.columns)} kolom.")

# 1.2 Membuat fungsi pembersih teks yang robust
def clean_text(text):
    """Fungsi untuk membersihkan teks dari HTML, URL, karakter spesial, dan spasi berlebih."""
    # Menangani jika input bukan string (misalnya, float NaN)
    if not isinstance(text, str):
        return ""

    # Menghapus tag HTML
    text = re.sub(r'<.*?>', '', text)
    # Menghapus URL
    text = re.sub(r'http\S+|www\S+|https\S+', '', text, flags=re.MULTILINE)
    # Menghapus karakter newline dan tab, menggantinya dengan spasi
    text = text.replace('\n', ' ').replace('\r', ' ').replace('\t', ' ')
    # Menghapus karakter spesial, hanya menyisakan huruf, angka, dan beberapa tanda baca dasar
    text = re.sub(r'[^a-zA-Z0-9\s.,\'!?-]', '', text)
    # Menghapus spasi berlebih di awal, akhir, dan tengah
    text = re.sub(r'\s+', ' ', text).strip()
    return text

# Menerapkan fungsi pembersih ke kolom 'Review Pros' dan 'Review Cons'
df_cleaned['Review Pros'] = df_cleaned['Review Pros'].apply(clean_text)
df_cleaned['Review Cons'] = df_cleaned['Review Cons'].apply(clean_text)
print("   - Teks pada kolom 'Review Pros' dan 'Review Cons' telah dibersihkan.")

# 2. HANDLING MISSING ENTRIES
print("\nMemulai proses Handling Missing Entries...")
baris_sebelum_handling = len(df_cleaned)

# Menghapus baris di mana KEDUA kolom 'Review Pros' DAN 'Review Cons' kosong setelah dibersihkan
# Ini memastikan ulasan tersebut benar-benar tidak memiliki konten teks.
df_cleaned = df_cleaned[~((df_cleaned['Review Pros'] == '') & (df_cleaned['Review Cons'] == ''))]

baris_setelah_handling = len(df_cleaned)
print(f"   - Jumlah baris sebelum: {baris_sebelum_handling}")
print(f"   - Jumlah baris sesudah: {baris_setelah_handling}")
print(f"   - Total baris kosong yang dihapus: {baris_sebelum_handling - baris_setelah_handling}")

# 3. DUPLICATE ELIMINATION
print("\nMemulai proses Duplicate Elimination...")
baris_sebelum_dedup = len(df_cleaned)

# Menghapus baris duplikat berdasarkan konten ulasan dan tanggalnya
# Ini adalah cara yang robust untuk memastikan ulasan yang sama persis tidak dihitung dua kali
kolom_untuk_duplikat = ['Review Date', 'Review Pros', 'Review Cons']
df_cleaned.drop_duplicates(subset=kolom_untuk_duplikat, inplace=True, keep='first')

baris_setelah_dedup = len(df_cleaned)
print(f"   - Jumlah baris sebelum: {baris_sebelum_dedup}")
print(f"   - Jumlah baris sesudah: {baris_setelah_dedup}")
print(f"   - Total baris duplikat yang dihapus: {baris_sebelum_dedup - baris_setelah_dedup}")

# 5. MENYIMPAN HASIL
# Menyimpan dataframe yang sudah bersih ke file CSV baru
nama_file_output = 'tokopedia_reviews_cleaned.csv'
df_cleaned.to_csv(nama_file_output, index=False, encoding='utf-8-sig')

print(f"\n🎉 Proses selesai! Dataset yang bersih telah disimpan sebagai '{nama_file_output}'.")
print(f"Bentuk data akhir: {df_cleaned.shape[0]} baris, {df_cleaned.shape[1]} kolom.")
print("\n5 baris pertama dari data yang sudah bersih:")
print(df_cleaned.head())

### Merapikan format tanggal

In [ ]:
import pandas as pd
import sys

# 0. Load dataset
nama_file_input = 'tokopedia_reviews_new.csv'
nama_file_output = 'tokopedia_reviews_date_cleaned.csv' # Nama file output baru

# 1. Pembersihan format tanggal
try:
    # Load data dari file CSV
    df = pd.read_csv(nama_file_input)
    print(f"✅ File '{nama_file_input}' berhasil dimuat. Total baris: {len(df)}")

    # Memeriksa apakah kolom 'Review Date' ada (menggunakan nama kolom yang dikoreksi)
    if 'Review Date' in df.columns:
        print("\nMemproses kolom 'Review Date'...")

        # 1. Mengubah kolom 'Review Date' ke format datetime
        # errors='coerce' akan mengubah nilai yang tidak bisa di-parse menjadi NaT (Not a Time)
        df['Review Date'] = pd.to_datetime(df['Review Date'], errors='coerce')

        # 2. Menghapus baris yang memiliki nilai NaT setelah konversi (jika ada data tanggal yang rusak)
        initial_rows = len(df)
        df.dropna(subset=['Review Date'], inplace=True) # Menggunakan nama kolom yang dikoreksi
        if len(df) < initial_rows:
            print(f"⚠️ {initial_rows - len(df)} baris dengan format tanggal tidak valid telah dihapus.")

        # 3. Memformat ulang kolom 'Review Date' agar hanya menampilkan tanggal (DD-MM-YYYY)
        df['Review Date'] = df['Review Date'].dt.strftime('%d-%m-%Y') # Menggunakan nama kolom yang dikoreksi
        print("✅ Kolom 'Review Date' berhasil dirapikan (format DD-MM-YYYY).")

        # Menampilkan beberapa baris pertama dengan kolom tanggal yang sudah dirapikan
        print("\n--- Contoh Data Setelah Dirapikan ---")
        # Pastikan Anda memilih kolom yang ingin ditampilkan, termasuk 'Review Date'
        print(df[['Review Date', 'Review Pros', 'Review Cons']].head())

        # Menyimpan DataFrame yang sudah dirapikan ke file CSV baru
        df.to_csv(nama_file_output, index=False, encoding='utf-8-sig')
        print(f"\n✅ Data yang dirapikan telah disimpan ke file '{nama_file_output}'.")

    else:
        print(f"❌ Kolom 'Review Date' tidak ditemukan dalam file '{nama_file_input}'.")
        print("Pastikan nama kolom sudah benar (case-sensitive, termasuk spasi jika ada).")

except FileNotFoundError:
    print(f"❌ ERROR: File '{nama_file_input}' tidak ditemukan.")
    print("Pastikan file CSV berada di direktori yang sama dengan skrip Python ini.")
    sys.exit()
except Exception as e:
    print(f"❌ Terjadi kesalahan saat memproses file: {e}")
    sys.exit()

✅ File 'tokopedia_reviews_new.csv' berhasil dimuat. Total baris: 1795

Memproses kolom 'Review Date'...
⚠️ 3 baris dengan format tanggal tidak valid telah dihapus.
✅ Kolom 'Review Date' berhasil dirapikan (format DD-MM-YYYY).

--- Contoh Data Setelah Dirapikan ---
  Review Date                                        Review Pros  \
0  19-06-2025                   No Comment No Comment No Comment   
1  16-06-2025  Collaborative team, supporting each other with...   
2  14-06-2025             good people culture growth opportunity   
3  08-06-2025           you will get bonus when you perform well   
4  07-06-2025                 have some future plan for employer   

                                  Review Cons  
0             There is no cons on the company  
1               Unstable future due to merger  
2  fast pace lots of initiatives at same time  
3                    the salary is pretty low  
4                i do not thing there is cons  

✅ Data yang dirapikan telah disimpan 